In [1]:
import pandas as pd
import numpy as np
import fiona
import matplotlib.pyplot as plt
import rasterio
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
from glob import glob

%matplotlib inline

In [25]:
with fiona.open('data/extreme_pr/north_coast.shp') as shapefile:
    features = [feature["geometry"] for feature in shapefile]

In [26]:
polygon = features[0]

### Get a list of NetCDF files to process

In [37]:
data_dir = 'data/extreme_pr'
file_list = glob(os.path.join(data_dir, '*.nc'))
print(file_list)

['data/extreme_pr/pr_historical_MIROC5_1976.nc', 'data/extreme_pr/pr_historical_MIROC5_1985.nc', 'data/extreme_pr/pr_historical_MIROC5_2004.nc', 'data/extreme_pr/pr_historical_MIROC5_1952.nc', 'data/extreme_pr/pr_historical_MIROC5_1995.nc', 'data/extreme_pr/pr_historical_MIROC5_1966.nc', 'data/extreme_pr/pr_historical_MIROC5_1972.nc', 'data/extreme_pr/pr_historical_MIROC5_1981.nc', 'data/extreme_pr/pr_historical_MIROC5_2000.nc', 'data/extreme_pr/pr_historical_MIROC5_1991.nc', 'data/extreme_pr/pr_historical_MIROC5_1962.nc', 'data/extreme_pr/pr_historical_MIROC5_1956.nc', 'data/extreme_pr/pr_historical_MIROC5_1973.nc', 'data/extreme_pr/pr_historical_MIROC5_1980.nc', 'data/extreme_pr/pr_historical_MIROC5_2001.nc', 'data/extreme_pr/pr_historical_MIROC5_1990.nc', 'data/extreme_pr/pr_historical_MIROC5_1963.nc', 'data/extreme_pr/pr_historical_MIROC5_1957.nc', 'data/extreme_pr/pr_historical_MIROC5_1977.nc', 'data/extreme_pr/pr_historical_MIROC5_1984.nc', 'data/extreme_pr/pr_historical_MIROC5_2

In [38]:
crs = 'EPSG:4326'
df_list = []

def run():
    for file in file_list:
        with rasterio.open(file) as src:
            # Get src name and parse climvar, scenario, model, year
            name = os.path.splitext(os.path.basename(src.name))[0]
            climvar, scenario, model, year = name.split('_')
            print('Processing', year, '...')

            # Get projection and metadata info
            transform, width, height = calculate_default_transform(
            crs, crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': crs,
                'transform': transform,
                'width': width,
                'height': height,
                'driver': 'GTiff',
            })

            # Reproject to geotiff
            with rasterio.open('data/extreme_pr/geotiff_wgs84.tif', 'w', **kwargs) as dst:
                for i in range(1, src.count):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.nearest)

            dst.close()
            src.close()

            # Clip geotiff to polygon
            # Calculate average values of all grid cells within clipped raster for each timestep (daily)
            # Write out to a dataframe with additional columns
            with rasterio.open('data/extreme_pr/geotiff_wgs84.tif') as src:
                # Get nodata value
                nodata = src.nodata
                timesteps = src.count

                 # Create a datetime array of length equal to number of daily timesteps in src
                date = np.array((year + '-01-01'), dtype=np.datetime64)
                date_arr = date + np.arange(timesteps)

                 # Create an empty array to hold average value
                # of all grid cells intersected by polygon
                mean_arr = []

                # Mask src with a polygon 
                # if shapefile has multiple polygons, you need to nest this in a loop
                out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)

                # For each timestep
                for i in range(0, timesteps):
                    vals = out_image[i]
                    data = vals[vals != nodata] # Filter out nodata values
                    mean_arr.append(data.mean()) # Calculate mean

                df = pd.DataFrame({'date':date_arr, 'value':mean_arr})
                df['value'] = df['value'] * 86400 # Convert values to mm
                #annual_total = np.sum(df['value'])
                #df['annual_total'] = annual_total
                #df['proportion'] = df['value'] / annual_total
                #df['model'] = model
                #df['scenario'] = scenario
                #df = df.sort_values(by='value')
                #df['cumsum'] = df.value.cumsum()
                df_list.append(df) # Add dataframe to list

            src.close()
    
    # Return a new dataframe with daily means for polygon from all source files      
    data = pd.concat(df_list)
    data = data.set_index('date')
    return data

In [39]:
data = run()
data.to_csv('data/csv/pr_historical_MIROC5.csv')

Processing 1976 ...
Processing 1985 ...
Processing 2004 ...
Processing 1952 ...
Processing 1995 ...
Processing 1966 ...
Processing 1972 ...
Processing 1981 ...
Processing 2000 ...
Processing 1991 ...
Processing 1962 ...
Processing 1956 ...
Processing 1973 ...
Processing 1980 ...
Processing 2001 ...
Processing 1990 ...
Processing 1963 ...
Processing 1957 ...
Processing 1977 ...
Processing 1984 ...
Processing 2005 ...
Processing 1953 ...
Processing 1994 ...
Processing 1967 ...
Processing 1958 ...
Processing 1978 ...
Processing 1968 ...
Processing 1979 ...
Processing 1969 ...
Processing 1959 ...
Processing 1989 ...
Processing 1999 ...
Processing 1988 ...
Processing 1998 ...
Processing 1983 ...
Processing 1970 ...
Processing 2002 ...
Processing 1960 ...
Processing 1993 ...
Processing 1954 ...
Processing 1987 ...
Processing 1974 ...
Processing 1950 ...
Processing 1964 ...
Processing 1997 ...
Processing 1986 ...
Processing 1975 ...
Processing 1951 ...
Processing 1965 ...
Processing 1996 ...


### Livneh Data

In [30]:
data_dir = 'data/livneh'
file_list = glob(os.path.join(data_dir, '*.nc'))
print(file_list)

['data/livneh/livneh_1977_6.nc', 'data/livneh/livneh_1995_6.nc', 'data/livneh/livneh_2003_12.nc', 'data/livneh/livneh_1957_6.nc', 'data/livneh/livneh_1983_10.nc', 'data/livneh/livneh_2002_9.nc', 'data/livneh/livneh_1992_12.nc', 'data/livneh/livneh_2005_2.nc', 'data/livneh/livneh_1981_11.nc', 'data/livneh/livneh_1957_11.nc', 'data/livneh/livneh_1998_5.nc', 'data/livneh/livneh_1955_4.nc', 'data/livneh/livneh_1991_5.nc', 'data/livneh/livneh_1973_3.nc', 'data/livneh/livneh_1988_12.nc', 'data/livneh/livneh_1975_3.nc', 'data/livneh/livneh_1997_2.nc', 'data/livneh/livneh_1962_3.nc', 'data/livneh/livneh_1970_3.nc', 'data/livneh/livneh_1952_8.nc', 'data/livneh/livneh_1967_7.nc', 'data/livneh/livneh_1963_12.nc', 'data/livneh/livneh_2003_5.nc', 'data/livneh/livneh_1974_12.nc', 'data/livneh/livneh_1963_11.nc', 'data/livneh/livneh_1981_7.nc', 'data/livneh/livneh_1974_11.nc', 'data/livneh/livneh_1968_5.nc', 'data/livneh/livneh_1969_9.nc', 'data/livneh/livneh_1991_6.nc', 'data/livneh/livneh_1995_5.nc

In [31]:
crs = 'EPSG:4326'
df_list = []

def run_livneh():
    for file in file_list:
        # Get src name and parse climvar, scenario, model, year
        name = os.path.splitext(os.path.basename(file))[0]
        model, year, month = name.split('_')
        subdataset = 'netcdf:' + file + ':Prec'
        with rasterio.open(subdataset) as src:
            print('Processing', year, month, '...')
            
            # Get projection and metadata info
            transform, width, height = calculate_default_transform(
            crs, crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': crs,
                'transform': transform,
                'width': width,
                'height': height,
                'driver': 'GTiff',
            })

            # Reproject to geotiff
            with rasterio.open('data/livneh/geotiff_wgs84.tif', 'w', **kwargs) as dst:
                for i in range(1, src.count):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.nearest)

            dst.close()
            src.close()

            # Clip geotiff to polygon
            # Calculate average values of all grid cells within clipped raster for each timestep (daily)
            # Write out to a dataframe with additional columns
            with rasterio.open('data/livneh/geotiff_wgs84.tif') as src:
                # Get nodata value
                nodata = src.nodata
                timesteps = src.count

                 # Create a datetime array of length equal to number of daily timesteps in src
                date = np.array((year + '-' + month.zfill(2) + '-01'), dtype=np.datetime64)
                print(date)
                date_arr = date + np.arange(timesteps)

                 # Create an empty array to hold average value
                # of all grid cells intersected by polygon
                mean_arr = []

                # Mask src with a polygon 
                # if shapefile has multiple polygons, you need to nest this in a loop
                out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)

                # For each timestep
                for i in range(0, timesteps):
                    vals = out_image[i]
                    data = vals[vals != nodata] # Filter out nodata values
                    mean_arr.append(data.mean()) # Calculate mean

                df = pd.DataFrame({'date':date_arr, 'value':mean_arr})
                df_list.append(df) # Add dataframe to list

            src.close()
    
    # Return a new dataframe with daily means for polygon from all source files      
    data = pd.concat(df_list)
    data = data.set_index('date')
    return data

In [32]:
livneh = run_livneh()
livneh.to_csv('data/csv/livneh_pr.csv')

Processing 1977 6 ...
1977-06-01
Processing 1995 6 ...
1995-06-01
Processing 2003 12 ...
2003-12-01
Processing 1957 6 ...
1957-06-01
Processing 1983 10 ...
1983-10-01
Processing 2002 9 ...
2002-09-01
Processing 1992 12 ...
1992-12-01
Processing 2005 2 ...
2005-02-01
Processing 1981 11 ...
1981-11-01
Processing 1957 11 ...
1957-11-01
Processing 1998 5 ...
1998-05-01
Processing 1955 4 ...
1955-04-01
Processing 1991 5 ...
1991-05-01
Processing 1973 3 ...
1973-03-01
Processing 1988 12 ...
1988-12-01
Processing 1975 3 ...
1975-03-01
Processing 1997 2 ...
1997-02-01
Processing 1962 3 ...
1962-03-01
Processing 1970 3 ...
1970-03-01
Processing 1952 8 ...
1952-08-01
Processing 1967 7 ...
1967-07-01
Processing 1963 12 ...
1963-12-01
Processing 2003 5 ...
2003-05-01
Processing 1974 12 ...
1974-12-01
Processing 1963 11 ...
1963-11-01
Processing 1981 7 ...
1981-07-01
Processing 1974 11 ...
1974-11-01
Processing 1968 5 ...
1968-05-01
Processing 1969 9 ...
1969-09-01
Processing 1991 6 ...
1991-06-01


1986-03-01
Processing 2001 9 ...
2001-09-01
Processing 1989 10 ...
1989-10-01
Processing 1969 2 ...
1969-02-01
Processing 2005 11 ...
2005-11-01
Processing 1978 7 ...
1978-07-01
Processing 1967 2 ...
1967-02-01
Processing 1999 7 ...
1999-07-01
Processing 1997 10 ...
1997-10-01
Processing 1965 1 ...
1965-01-01
Processing 1996 9 ...
1996-09-01
Processing 2006 11 ...
2006-11-01
Processing 1990 9 ...
1990-09-01
Processing 1981 12 ...
1981-12-01
Processing 1966 11 ...
1966-11-01
Processing 2001 12 ...
2001-12-01
Processing 1996 1 ...
1996-01-01
Processing 1985 5 ...
1985-05-01
Processing 1988 5 ...
1988-05-01
Processing 1984 3 ...
1984-03-01
Processing 2004 4 ...
2004-04-01
Processing 1961 10 ...
1961-10-01
Processing 1955 1 ...
1955-01-01
Processing 1984 7 ...
1984-07-01
Processing 1961 5 ...
1961-05-01
Processing 1988 8 ...
1988-08-01
Processing 1997 8 ...
1997-08-01
Processing 2003 1 ...
2003-01-01
Processing 2004 11 ...
2004-11-01
Processing 1968 2 ...
1968-02-01
Processing 1965 6 ...
1

2002-04-01
Processing 1999 4 ...
1999-04-01
Processing 1954 3 ...
1954-03-01
Processing 1999 1 ...
1999-01-01
Processing 2001 3 ...
2001-03-01
Processing 1970 12 ...
1970-12-01
Processing 2003 3 ...
2003-03-01
Processing 1996 4 ...
1996-04-01
Processing 1999 9 ...
1999-09-01
Processing 1972 5 ...
1972-05-01
Processing 1973 7 ...
1973-07-01
Processing 1982 11 ...
1982-11-01
Processing 1950 6 ...
1950-06-01
Processing 1971 10 ...
1971-10-01
Processing 2002 5 ...
2002-05-01
Processing 2005 7 ...
2005-07-01
Processing 1962 2 ...
1962-02-01
Processing 1979 10 ...
1979-10-01
Processing 1980 10 ...
1980-10-01
Processing 1963 2 ...
1963-02-01
Processing 1978 11 ...
1978-11-01
Processing 1960 1 ...
1960-01-01
Processing 1988 9 ...
1988-09-01
Processing 1987 8 ...
1987-08-01
Processing 1959 10 ...
1959-10-01
Processing 1965 10 ...
1965-10-01
Processing 2000 8 ...
2000-08-01
Processing 1989 7 ...
1989-07-01
Processing 1973 8 ...
1973-08-01
Processing 1951 4 ...
1951-04-01
Processing 1959 9 ...
19

In [33]:
livneh

,value
date,
1977-06-01,0.111161
1977-06-02,0.010514
1977-06-03,0.086673
1977-06-04,0.185156
1977-06-05,0.039974
1977-06-06,0.310144
1977-06-07,1.668031
1977-06-08,2.256704
1977-06-09,1.022243
